# imports

In [1]:
import torch
from transformers import *
from fastai2.text.all import *

from blurr.data.all import *
from blurr.modeling.all import *

# Data

In [2]:
path = untar_data(URLs.IMDB_SAMPLE)

model_path = Path('models')
imdb_df = pd.read_csv(path/'texts.csv')

# Get your hugs huggingface objects

In [3]:
task = HF_TASKS_AUTO.SequenceClassification

pretrained_model_name = "bert-base-uncased"
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR_MODEL_HELPER.get_hf_objects(pretrained_model_name,  task=task)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# Build your bricksbricksbricks DataBlock bricksbricksbricks and your DataLoaders

In [4]:
# single input
blocks = (HF_TextBlock(hf_arch=hf_arch, hf_tokenizer=hf_tokenizer), CategoryBlock)

dblock = DataBlock(blocks=blocks, 
                   get_x=ColReader('text'), get_y=ColReader('label'), 
                   splitter=ColSplitter(col='is_valid'))

dls = dblock.dataloaders(imdb_df, bs=4)
dls.show_batch(hf_tokenizer=hf_tokenizer, max_n=2)

,text,category
0,"un - bleeping - believable! meg ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. hard to believe she was the producer on this dog. plus kevin kline : what kind of suicide trip has his career been on? whoosh... banzai!!! finally this was directed by the guy who did big chill? must be a replay of jonestown - hollywood style. wooofff!",negative
1,"a below average looking video game is turned into some sort of conspiracy to have the next terrorist discovered in the usa backyard. welcome to the lunacy of cheaply made direct to video movies. its full of no - name actors and actresses with little valuable plot. < br / > < br / > anyway, this strange game goes on and our "" hero "" bets real money and does good at it. it is sort of like gambling, except the gambling part is gone and it sucks. instead its an online game with little real value and you get authorities on your tail if you do good. < br / > < br / > what makes it even stranger is that two strange computer programs battle it out somehow and all is saved in the end. i will leave the viewer to see how it all comes to fruition. < br / > < br / > overall, not even worth a $ 1 rental. borrow it, please. "" d - """,negative


# ... and train steam_locomotive

In [5]:
%%time
#slow
model = HF_BaseModelWrapper(hf_model)

learn = Learner(dls, 
                model,
                opt_func=partial(Adam, decouple_wd=True),
                loss_func=CrossEntropyLossFlat(),
                metrics=[accuracy],
                cbs=[HF_BaseModelCallback],
                splitter=hf_splitter)

learn.create_opt() 
learn.freeze()

learn.fit_one_cycle(3, lr_max=1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.677201,0.670366,0.610000,00:20
1,0.633752,0.666956,0.580000,00:19
2,0.617348,0.609058,0.715000,00:20


CPU times: user 56.6 s, sys: 1.04 s, total: 57.7 s
Wall time: 1min


In [6]:
%%time
learn.show_results(hf_tokenizer=hf_tokenizer, max_n=2)

,text,category,target
0,"this very funny british comedy shows what might happen if a section of london, in this case pimlico, were to declare itself independent from the rest of the uk and its laws, taxes & post - war restrictions. merry mayhem is what would happen. < br / > < br / > the explosion of a wartime bomb leads to the discovery of ancient documents which show that pimlico was ceded to the duchy of burgundy centuries ago, a small historical footnote long since forgotten. to the new burgundians, however, this is an unexpected opportunity to live as they please, free from any interference from whitehall. < br / > < br / > stanley holloway is excellent as the minor city politician who suddenly finds himself leading one of the world's tiniest nations. dame margaret rutherford is a delight as the history professor who sides with pimlico. others in the stand - out cast include hermione baddeley, paul duplis, naughton wayne, basil radford & sir michael hordern. < br / > < br / > welcome to burgundy!",positive,positive
1,"about the baby : why wasn't big brother assuming he'd be hungry for a bottle or some nourishment or a diaper change? he should have been screaming non - stop after that many hours without care. definitely stupid to take the baby from a safe place when he didn't need to. < br / > < br / > and why was the road miraculously clear whenever anyone wanted to drive somewhere? didn't any uprooted trees fall on the roads and block them? < br / > < br / > i can't imagine the cops at the roadblock not immediately following after any young person who would crash it, especially when they said it was dangerous to go there. < br / > < br / > that being said, it was nice to have a movie children could safely watch, for a change.",negative,negative


CPU times: user 129 ms, sys: 30 µs, total: 130 ms
Wall time: 131 ms
